In [1]:
OPENAI_API_KEY=""

In [2]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

In [3]:
MAX_TOKEN = 6000
MAX_CHAT = 10
SCENARIO_ID = 1
PERSONA = 'submissive_people'
AGENT_NAME = 'sparky'

In [4]:
from enum import Enum
from pydantic import BaseModel

class UserAction(str, Enum):
    say = "say"
    leave = "leave"

class UserResponse(BaseModel):
    action: UserAction
    answer: str

In [17]:
import json
class Person:
    def __init__(self, scenarioID: int, persona: str):

        scenarioFile = open(f'./scenario/{scenarioID}.txt', 'r')
        scenario = scenarioFile.read()
        scenarioFile.close()

        personaFile = open(f'./persona/{persona}.txt', 'r')
        persona = personaFile.read()
        personaFile.close()

        example = "{action: say, answer: 'I feel tired'}, {action: leave, answer: 'I am leaving'}"
        systemPrompt = f'You are a user talking to AI APP which can help you deal with your problem during break time. \
                            This is your persona: {persona}\
                            Please play the role according to the scenario: {scenario}\
                            Use Action → Answer structure for responses.\
                            Available Actions:\
                            1. say: respond base on persona and scenario\
                            2. leave: leave the chat when you think the conversation is over, no need to continue\
                            Example: {example}'
        # print(systemPrompt)

        self.messages = [
            {'role': 'system', 'content': systemPrompt}, 
        ]
        self.leaveChat = False
    
    def say(self):
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=self.messages,
            response_format={
                'type': 'json_schema',
                'json_schema': 
                    {
                        "name":"whocares", 
                        "schema": UserResponse.model_json_schema()
                    }
            }
        )

        message = response.choices[0].message.content
        self.messages.append({'role': 'assistant', 'content': message})

        # str to dict
        message = json.loads(message)
        self.leaveChat = (message['action'] == 'leave')

        info = {
            'token': response.usage.total_tokens,
        }
        
        return message['answer'], info
    
    def listen(self, message: str):
        self.messages.append({'role': 'user', 'content': message})

        

In [6]:
test = Person(SCENARIO_ID, PERSONA)
ans = test.say()
print(ans)

("What does this app do? I'm not really sure how to get started.", {'token': 317})


In [7]:
class sparkyAction(str, Enum):
    call_bruno = "call_bruno"
    call_bizy = "call_bizy"
    ask_more = "ask_more"
    introduce_bruno = "introduce_bruno"
    introduce_bizy = "introduce_bizy"
    advise = "advise"

class AgentResponse(BaseModel):
    action: sparkyAction
    answer: str

In [8]:
import time
import json

class Animal:
    def __init__(self, name: str):

        self.agent_name = name
        agentFile = open(f'./agent/{name}.txt', 'r')
        agentPrompt = agentFile.read()
        agentFile.close()

        # create assistant and thread
        self.assistant = client.beta.assistants.create(
            name = self.agent_name,
            instructions = agentPrompt,
            model="gpt-4o-mini",
            response_format={
                'type': 'json_schema',
                'json_schema': 
                    {
                        "name":"whocares", 
                        "schema": AgentResponse.model_json_schema()
                    }
            }
        )
        self.thread = client.beta.threads.create()
        self.user_message = 'hello'

    def say(self):

        prompt = client.beta.threads.messages.create(
            thread_id = self.thread.id,
            role = "user",
            content = self.user_message
        )

        run = client.beta.threads.runs.create_and_poll(
            thread_id=self.thread.id,
            assistant_id=self.assistant.id,
        )

        while True:
            runData = client.beta.threads.runs.retrieve(
                thread_id=self.thread.id,
                run_id=run.id
            )

            if runData.status == 'completed': 
                response = client.beta.threads.messages.list(
                    thread_id=self.thread.id
                )
                message = json.loads(response.data[0].content[0].text.value)

                info = {
                    'token': runData.usage.total_tokens,
                    'action': message['action']
                }
                return message['answer'], info

            else:
                print("runData.status")
                time.sleep(2) 


    def listen(self, message: str):
        self.user_message = message
    

In [9]:
test = Animal(name = AGENT_NAME)
test.say()

('Hi there! What’s on your mind today? Are you feeling stressed, anxious, or maybe having trouble staying focused?',
 {'token': 525, 'action': 'ask_more'})

In [10]:
test.listen('i feel tired')

In [11]:
test.say()

('Sounds like you could use a break! Let me introduce you to Bruno, a bear who’s a mindfulness specialist. Want to meet him and discover some relaxing techniques?',
 {'token': 580, 'action': 'introduce_bruno'})

In [12]:
from pydantic import BaseModel

class EvaluatorResponse(BaseModel):
    accuracy: int
    practicality: int

In [13]:
import json
class Evaluator():
    def __init__(self) -> None:

        file = open(f'./evaluator/version0.txt', 'r')
        systemPrompt = file.read()
        file.close()
        self.messages = [
            {'role': 'system', 'content': systemPrompt}, 
        ]
        
    def evaluate(self, personMessage: str, animalMessage: str):
        self.messages.append({'role': 'user', 'content': f'User: {personMessage}\nAgent: {animalMessage}'})
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=self.messages,
            response_format={
                'type': 'json_schema',
                'json_schema': 
                    {
                        "name":"whocares", 
                        "schema": EvaluatorResponse.model_json_schema()
                    }
            }
        )

        message = json.loads(response.choices[0].message.content)
    
        return message['accuracy'], message['practicality']


In [14]:
test = Evaluator()
test.evaluate('I feel tired', 'Oh, that’s really sad.')

(2, 0)

In [ ]:
import pandas as pd
class Report():
    def __init__(self) -> None:
        self.finish = True
        self.chatHistory = []

    def addHistory(self,chat):
        self.chatHistory.append(chat)
    
    def generateReport(self):
        df = pd.DataFrame(self.chatHistory)
        df.to_csv('report.csv', index=False)

In [23]:
person = Person(scenarioID = SCENARIO_ID, persona = PERSONA)
animal = Animal(name= AGENT_NAME)
evaluator = Evaluator()
# report = Report()

totalToken = 0
totalChat = 0

while not person.leaveChat:
    # chat
    personMessage, personInfo = person.say()
    animal.listen(personMessage)
    animalMessage, animalInfo = animal.say()
    person.listen(animalMessage)

    print(f'User: {personMessage}\nAgent: {animalMessage}')
    print(f'user leave chat: {person.leaveChat}')

    # metrics
    accuracy, practicality = evaluator.evaluate(personMessage, animalMessage)
    print(f'Accuracy: {accuracy}, Practicality: {practicality}\n')
    

    # report
    # history = {
    #     'person_say': personMessage,
    #     'animal_action': animalInfo.action,
    #     'animal_say': animalMessage,
    #     'animal_token': animalInfo.token,
    #     'accuracy': accuracy,
    #     'practicality': practicality,
    # }

    # report.addHistory(history)
    totalChat += 1
    totalToken += animalInfo['token']

    if totalToken > MAX_TOKEN or totalChat > MAX_CHAT:
        # report.finish = False
        break



User: Hello! This app can help you with various tasks during your break time, like providing relaxation techniques, quick exercises, and managing stress. You can explore features like suggestions for mindfulness activities, guided meditations, or even quick trivia games to keep your mind engaged. Would you like me to guide you through the main features?
Agent: Oh, that sounds like a delightful break! Are you looking for relaxation techniques or some fun trivia games to de-stress? Let me help you find the best fit!
user leave chat: False
Accuracy: 9, Practicality: 9

User: Thank you for offering to help! I think relaxation techniques would be a great way to start. Could you show me some options?
Agent: Absolutely! There's a wonderful bear named Bruno who specializes in relaxation techniques. Would you like to meet him for some calming mindfulness tips?
user leave chat: False
Accuracy: 9, Practicality: 8

User: Yes, meeting Bruno sounds lovely! I would love to get some calming mindfulnes

KeyboardInterrupt: 

In [ ]:
report.generateReport()

In [18]:
print(totalToken, totalChat)

0 0
